In [1]:
import pandas as pd
import numpy as np

In [13]:
file_path = '../data/complete.csv'

In [15]:
chunk_size = 10000
chunks = []

In [17]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size, on_bad_lines='skip'):
    expected_columns = 11
    chunk = chunk[chunk.apply(lambda x: len(x) == expected_columns, axis=1)]

    chunks.append(chunk)
    

In [18]:
df = pd.concat(chunks, ignore_index=True)

In [19]:
df.columns = [
    'datetime', 'city', 'state', 'country', 'shape',
    'duration (seconds)', 'duration (hours/min)', 'comments',
    'date posted', 'latitude', 'longitude'
]

In [20]:
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611


In [21]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

In [22]:
df['date posted'] = pd.to_datetime(df['date posted'], errors='coerce').dt.date

In [23]:
df['duration (seconds)'] = pd.to_numeric(df['duration (seconds)'], errors='coerce')

In [24]:
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

In [25]:
df['city'] = df['city'].str.strip().str.title()
df['state'] = df['state'].str.strip().str.upper()
df['country'] = df['country'].str.strip().str.upper()

In [26]:
df['duration (hours/min)'] = df['duration (hours/min)'].str.strip()

In [27]:
df.dropna(subset=['datetime', 'city', 'country'], inplace=True)

In [28]:
df.drop_duplicates(inplace=True)

In [29]:
df = df[(df['latitude'].between(-90, 90)) & (df['longitude'].between(-180, 180))]

In [30]:
df.rename(columns={
    'datetime': 'datetime',
    'city': 'city',
    'state': 'state',
    'country': 'country',
    'shape': 'shape',
    'duration (seconds)': 'duration_seconds',
    'duration (hours/min)': 'duration_hours_min',
    'comments': 'comments',
    'date posted': 'date_posted',
    'latitude': 'latitude',
    'longitude': 'longitude'
}, inplace=True)

In [31]:
df.head()

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,1949-10-10 20:30:00,San Marcos,TX,US,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
2,1955-10-10 17:00:00,Chester (Uk/England),NaN,GB,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,Edna,TX,US,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,Kaneohe,HI,US,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611
5,1961-10-10 19:00:00,Bristol,TN,US,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.595000,-82.188889


In [33]:
cleaned_file_path = '../data/cleaned_ufo_sightings.csv'
df.to_csv(cleaned_file_path, index=False)
print(f"Cleaned data has been saved to {cleaned_file_path}")


Cleaned data has been saved to ../data/cleaned_ufo_sightings.csv
